Robot Localization with Python and Particle Filters
===================================================

Import libraries and load map.

In [2]:
 
#robot doesn't know where it landed or which direction it is pointed at 
#all it has is a map of the planet's surface that shows the terrain height 
#it also has a sensor that measures its own elevation 
#using only those two tools we want the robot to figure out 
#where it is and keep trackign itself as it moves around 
 #in the demo the green and thered dot were far so he didn't have a clue
#but as it kep running nthe two dots got closer meaning he started to figure out its position 
import numpy as np
import cv2
map=cv2.imread("map.png",0)
#the image is stored as 2 dimensional numpy array brightness 0-255
HEIGHT,WIDTH=map.shape
print(map)
#initialize robots position 
rx,ry,rtheta=(WIDTH/4,HEIGHT/4,0)
#the coordinat system we gonna use the open convention from open cv 
#

'pip' is not recognized as an internal or external command,
operable program or batch file.


ModuleNotFoundError: No module named 'numpy'

Map coordinate system

![title](images/coords.png)

CAUTION: The terrain height at X,Y coordinates is map(Y,X).

Read keyboard input.

In [ ]:
STEP=5
TURN = np.radians(25)
def get_input():
    fwd=0
    turn=0
    halt=False
    k=cv2.waitKey(0)
    if k==82:
        fwd=STEP
    elif k==83:
        turn=TURN
    elif k==81:
        turn=-TURN
    else:
        halt=True
    return fwd, turn, halt
#can move robot around with the arrow keys 

Move the robot, with Gausssian noise.

![title](images/gaussian.png)

greek line at the center is greek letter mu 
it is the forward value that we want to command the robot 
but the distance it actually moves maybe a bit more or maybe a bit less 
#the blue curve here represents the values we can get
-----------------------------

Sigma is the standard deviation 

In [ ]:
#now we want to update the robot position according to the keyboard command 
#In the real world the terrrain is uneven robot control isn't perfect 
#there's some uncertainty how the robot will move according to our command 
#we can simulate this by adding gaussian noise to the stepping and steering values 
SIGMA_STEP=0.5
SIGMA_TURN = np.radians(5)

def move_robot(rx, ry, rtheta, fwd, turn):
    fwd_noisy=np.random.normal(fwd,SIGMA_STEP,1)
    rx +=fwd_noisy *  np.cos(rtheta)
    ry +=fwd_noisy *  np.sin(rtheta)
    print("fwd_noisy",fwd_noisy)
    turn_noisy=np.random.normal(turn,SIGMA_TURN,1)
    rtheta +=turn_noisy
    print("turn_noisy", np.degrees(turn_noisy))
    
    return rx, ry, rtheta

Initialize particle cloud.

In [ ]:
#particle filters
#we want the robot to figure out its own position 
#using its terrain map and its sensor for measuring elevation 
#simple if you can take a measurement and then find all the pixels on the map that are closer to that elevation 
#it would reveal a lot of matching locations on the map 
#if the robot moves it had to take another measurement then recheck the whole map all over again 
#this method doesn't converge to a single location need a more clever way 
#Idea: 
#place a few thousands particles all over this map random position random orientations 
#each particle represent a hypothesis about where the robot might be 
#when we give a command to the robot we will try to move particles in same way 
#then we would try to improve our cloud of particles +give estimate that converges to the robot location 

NUM_PARTICLES=3000
def init():
    particles=np.random.rand(NUM_PARTICLES,3)
    particles*=np.array((WIDTH,HEIGHT,np.radians(360)))
    return particles

Move the particles. like robot according to particle command 

In [ ]:
def move_particles(particles, fwd, turn):
#we aren't going to add any noise in the movement 
#one column at a time x position 
#: to select all rows of a a specific column 
    particles [:,0]+= fwd * np.cos(particles[:,2])
#whole column at a time 
#y position  2nd column
    particles [:,1]+= fwd * np.sin(particles[:,2])
#want to increment the particles heading 
    particles[:,2] += turn 
#want to be careful we don't want particles moving ot f the edges of the map 
    particles[:,0]= np.clip(particles[:,0],0.0,WIDTH-1)
    particles[:,1]= np.clip(particles[:,1],0.0,HEIGHT-1)
    return particles

Get value from robot's sensor.

In [ ]:
#how to robot senses its own elevation and decide which of these particles are the best
#hypotheses for our robot's position 
SIGMA_SENSOR=2
def sense(x, y, noisy=False):
    x=init(x)
    y=init(y)
    if noisy: 
        return np.random.normal(map[y,x],SIGMA_SENSOR,1)
    
    
    return map[y,x]

Compute particle weights.

In [ ]:
#assign weights to the particles higher weights means elevation is a closer match
#to the robot sensor measurement 
def compute_weights(particles, robot_sensor): 
    errors= np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        elevation = sense(particles[i,0],particles[i,1],noisy=False )
        #now what we put in the errors array will be the difference 
        #difference between the robot sensor value and the particle elevation 
        errors[i]= abs(robot_sensor -  elevation) #not interested in positive and negative 
#assign weights to the particles  go up as the error goes down 
    weights = np.max(errors)-errors
    #we don't want the particles to pile up on the edges of the map it happens quite easily 
    #want to set the weights to 0 when a particle reaches the edge 
    weights[
        (particles[:,0]==0) #return a new array or true and false values  condition element-wise
        | 
        (particles[:,0]==WIDTH-1) #building an index for the weights array  
        |
        (particles[:,1]==0) #we use height instead of width for the y 
         |
        (particles[:,1]==HEIGHT-1)
    ]=0.0

      #CUBE THE WEIGHTS MAKE IT   work better by insreasing the sensitivity 
    weights = weights **3   

    return weights

Resample the particles.

In [ ]:
#use this weights to refine estimate of the robot state 
def resample(particles, weights):
#normalize the weights so they sum to 1 
    probabilities =weights /np.sum(weights)
#this will allow us to use them as a probability distribution over the particles 
#sample the particles according to this probability distribution 
    new_index=np.random.choice(NUM_PARTICLES,size=NUM_PARTICLES,p=probabilities)
    #return an array of row index numbers 
    #we can use this index to create a new particle array 
    particles=particles[new_index,:]
    #so we have ressampled the particles 
    #high weights get ressampled many times  low weights no 
    #this has improved the estimate of the robot state
    
    

    return particles

Add noise to the particles.

In [ ]:
SIGMA_POS=2
SIGMA_TURN=np.radians(10)

#add random variations Gaussian noise 
def add_noise(particles):
    noise = np.concatenate(
    (
        np.random.normal(0,SIGMA_POS,(NUM_PARTICLES,1)), #X 
        np.random.normal(0,SIGMA_POS,(NUM_PARTICLES,1)),#Y same 
        np.random.normal(0,SIGMA_TURN,(NUM_PARTICLES,1)), #theta different std deviation 
        
            
    ),
    #to tell it how to do the concatenation column-wise 
        axis=1
    )
    particles += noise 
    
    return particles

Display robot, particles and best guess.

In [ ]:
def display(map, rx, ry, particles):
    lmap = cv2.cvtColor(map, cv2.COLOR_GRAY2BGR)
    
    # Display particles
    if len(particles) > 0:
        for i in range(NUM_PARTICLES):
            cv2.circle(lmap, 
                       (int(particles[i,0]), int(particles[i,1])), 
                       1, 
                       (255,0,0), 
                       1)
        
    # Display robot
    cv2.circle(lmap, (int(rx), int(ry)), 5, (0,255,0), 10)

    # Display best guess
    if len(particles) > 0:
        px = np.mean(particles[:,0])
        py = np.mean(particles[:,1])
        cv2.circle(lmap, (int(px), int(py)), 5, (0,0,255), 5)

    cv2.imshow('map', lmap)

Main routine.

In [ ]:
particles = init()
while True:
    display(map, rx, ry, particles)
    fwd, turn, halt = get_input()
    if halt:
        break
    rx, ry, rtheta = move_robot(rx, ry, rtheta, fwd, turn)
    particles = move_particles(particles, fwd, turn)
    if fwd != 0:
        robot_sensor = sense(rx, ry, noisy=True)        
        weights = compute_weights(particles, robot_sensor)
        particles = resample(particles, weights)
        particles = add_noise(particles)
    
cv2.destroyAllWindows()                        


The particles are concentrating in smaller and smaller region until they finally lock onto the robot 
You may find you have to give this a few tries because we had a very simplistic implementation 
sometimes it doesn't quite lock on properly but the capability is there an there are refinements that could be made so that it pretty much locks on every time without fail 

Pretty Sophisticated algortihm and it's able to maintain the lock 

Learn a lot about an important problem in robotics called localization 
a robot landed on a far away planrt and using only a terrain map and a noisy sensor 
it located its own position and kept tracking it while moving 
Real world challenge that is uncertainty in the movement and in the sensor measurements 

Particle filters powerful soltion for computer vision artificial intelligence even finance 